In [1]:
import sys
import time
import numpy as np
import pickle
import importlib

In [2]:
from qiskit import Aer, IBMQ
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

/var/folders/vr/d82vcp297v78vk4sb0_nzzdc0000gn/T/ipykernel_6483/2243832710.py:2: DeprecationWarning: The qiskit.ignis package is deprecated and has been supersceded by the qiskit-experiments project. Refer to the migration guide: https://github.com/Qiskit/qiskit-ignis#migration-guide on how to migrate to the new project.
  from qiskit.ignis.mitigation.measurement import TensoredMeasFitter


In [3]:
min_size = 40
max_size = 105

In [4]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [5]:
import libs_qrem
importlib.reload(libs_qrem)
from libs_qrem import LeastNormFilter

In [6]:
xz_lnp_mitigator_list = []
for i, n in enumerate(range(min_size + 60, max_size + 1)):
    mit_pattern = [[j] for j in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[i + 60], mit_pattern=mit_pattern)
    xz_lnp_mitigator_list.append(LeastNormFilter(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of xz_lnp_mitigator_list: ", len(xz_lnp_mitigator_list))

size 100 finished
length of xz_lnp_mitigator_list:  6


In [7]:
zx_lnp_mitigator_list = []
for i, n in enumerate(range(min_size + 60, max_size + 1)):
    mit_pattern = [[j] for j in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[i + 60], mit_pattern=mit_pattern)
    zx_lnp_mitigator_list.append(LeastNormFilter(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of zx_lnp_mitigator_list: ", len(zx_lnp_mitigator_list))

size 100 finished
length of zx_lnp_mitigator_list:  6


In [9]:
lnp_hist_list = []
xz_lnp_total_times = []
zx_lnp_total_times = []
for i, n in enumerate(range(min_size + 60, max_size + 1)):
    print(n, "qubits")
    
    t1 = time.perf_counter()
    lnp_hist_list.append( xz_lnp_mitigator_list[i].apply(raw_hist_list[2 * i + 60]) )
    t2 = time.perf_counter()
    xz_lnp_total_times.append(t2 - t1)
    print("mitigation time (xz): ", t2 - t1, "s")
    
    t3 = time.perf_counter()
    lnp_hist_list.append( zx_lnp_mitigator_list[i].apply(raw_hist_list[2 * i + 1 + 60]) )
    t4 = time.perf_counter()
    zx_lnp_total_times.append(t4 - t3)
    print("mitigation time (zx): ", t4 - t3, "s")
    
    print()

100 qubits
mitigation time (xz):  9.538738969 s
mitigation time (zx):  10.204448985000006 s

101 qubits
mitigation time (xz):  10.648341267000006 s
mitigation time (zx):  11.062919594000007 s

102 qubits
mitigation time (xz):  11.263696632999995 s
mitigation time (zx):  11.117270216999998 s

103 qubits
mitigation time (xz):  11.173413772000004 s
mitigation time (zx):  11.361627388000002 s

104 qubits
mitigation time (xz):  11.342945192999991 s
mitigation time (zx):  11.234107018999993 s

105 qubits
mitigation time (xz):  11.08206006200001 s
mitigation time (zx):  11.275758731999986 s



In [10]:
with open("./pkls/lnp_hist_list.pkl", "wb") as f:
    pickle.dump(lnp_hist_list, f)

In [11]:
with open("./pkls/lnp_total_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_lnp_total_times, "zx": zx_lnp_total_times}, f)

In [13]:
xz_lnp_times = []
zx_lnp_times = []
for i, n in enumerate(range(min_size + 60, max_size)):
    xz_lnp_times.append( xz_lnp_mitigator_list[i].times() )
    zx_lnp_times.append( zx_lnp_mitigator_list[i].times() )

In [14]:
with open("./pkls/lnp_times.pkl", "wb") as f:
    pickle.dump({"xz": xz_lnp_times, "zx": zx_lnp_times}, f)

In [16]:
xz_lnp_mitigators = []
for i, n in enumerate(range(min_size + 60, max_size)):
    xz_lnp_mitigators.append({"indices_to_keys_vector":xz_lnp_mitigator_list[i].indices_to_keys_vector(),
                              # "mitigated_hist":xz_lnp_mitigator_list[i].mitigated_hist(),
                              "one_norm":xz_lnp_mitigator_list[i].exact_one_norm_of_reduced_inv_A(),
                              # "reduced_inv_A":xz_lnp_mitigator_list[i].reduced_inv_A(),
                              "sum_of_x":xz_lnp_mitigator_list[i].sum_of_x(),
                              "sum_of_x_hat":xz_lnp_mitigator_list[i].sum_of_x_hat(),
                              "sum_of_x_tilde":xz_lnp_mitigator_list[i].sum_of_x_tilde(),
                              # "times":xz_lnp_mitigator_list[i].times(),
                              "x_s":xz_lnp_mitigator_list[i].x_s(),
                              "x_hat":xz_lnp_mitigator_list[i].x_hat(),
                              "x_tilde":xz_lnp_mitigator_list[i].x_tilde(),
                              })
    print(i, "finished")

0 finished
1 finished
2 finished
3 finished
4 finished


In [17]:
zx_lnp_mitigators = []
for i, n in enumerate(range(min_size + 60, max_size)):
    zx_lnp_mitigators.append({"indices_to_keys_vector":zx_lnp_mitigator_list[i].indices_to_keys_vector(),
                              # "mitigated_hist":zx_lnp_mitigator_list[i].mitigated_hist(),
                              "one_norm":zx_lnp_mitigator_list[i].exact_one_norm_of_reduced_inv_A(),
                              # "reduced_inv_A":zx_lnp_mitigator_list[i].reduced_inv_A(),
                              "sum_of_x":zx_lnp_mitigator_list[i].sum_of_x(),
                              "sum_of_x_hat":zx_lnp_mitigator_list[i].sum_of_x_hat(),
                              "sum_of_x_tilde":zx_lnp_mitigator_list[i].sum_of_x_tilde(),
                              # "times":zx_lnp_mitigator_list[i].times(),
                              "x_s":zx_lnp_mitigator_list[i].x_s(),
                              "x_hat":zx_lnp_mitigator_list[i].x_hat(),
                              "x_tilde":zx_lnp_mitigator_list[i].x_tilde(),
                              })
    print(i, "finished")

0 finished
1 finished
2 finished
3 finished
4 finished


In [18]:
with open("./pkls/lnp_mitigator_list.pkl", "wb") as f:
    pickle.dump({"xz": xz_lnp_mitigators, "zx": zx_lnp_mitigators}, f)